# 🦠 Malaria Cell Detection — Train & Export to TF.js
**By Scott Antwi**

This notebook trains a CNN on the NIH Malaria Dataset and converts it to TensorFlow.js for the web app.

⚡ **Make sure GPU is enabled:** Runtime → Change runtime type → T4 GPU

## Step 1: Install Dependencies

In [ ]:
!pip install tensorflowjs -q
print('✅ Ready')

## Step 2: Download Dataset

In [ ]:
import os

# Download the NIH Malaria dataset
!wget -q https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip
!unzip -q cell_images.zip -d dataset

# Find the data path
DATA_PATH = None
for root, dirs, files in os.walk('dataset/'):
    if 'Parasitized' in dirs:
        DATA_PATH = root + '/'
        break

print(f'📂 Data path: {DATA_PATH}')
print(f'📊 Parasitized: {len(os.listdir(DATA_PATH + "Parasitized"))} images')
print(f'📊 Uninfected: {len(os.listdir(DATA_PATH + "Uninfected"))} images')

## Step 3: Load & Preprocess Images

In [ ]:
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

IMG_SIZE = 64

def load_images(folder, label):
    images, labels = [], []
    for filename in os.listdir(folder):
        try:
            img = Image.open(os.path.join(folder, filename))
            img = img.resize((IMG_SIZE, IMG_SIZE))
            images.append(np.array(img))
            labels.append(label)
        except:
            pass
    return images, labels

print('Loading infected cells...')
X_infected, y_infected = load_images(DATA_PATH + 'Parasitized/', 1)
print(f'  ✅ {len(X_infected)} infected images')

print('Loading healthy cells...')
X_healthy, y_healthy = load_images(DATA_PATH + 'Uninfected/', 0)
print(f'  ✅ {len(X_healthy)} healthy images')

# Combine and normalize
X = np.array(X_infected + X_healthy, dtype='float32') / 255.0
y = np.array(y_infected + y_healthy)

del X_infected, X_healthy, y_infected, y_healthy

print(f'\n📊 Total: {len(X)} images')
print(f'📊 Infected: {sum(y)} | Healthy: {len(y) - sum(y)}')

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'\n🔀 Train: {len(X_train)} | Test: {len(X_test)}')

## Step 4: Build & Train CNN

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Block 1
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D((2, 2)),

    # Block 2
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Block 3
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    # Classification head
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

print('\n🚀 Training...')
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.15,
    verbose=1
)

loss, accuracy = model.evaluate(X_test, y_test)
print(f'\n🎯 Test Accuracy: {round(accuracy * 100, 2)}%')
print(f'📉 Test Loss: {round(loss, 4)}')

## Step 5: Convert to TensorFlow.js

In [ ]:
import tensorflowjs as tfjs
import shutil

# Convert model
tfjs.converters.save_keras_model(model, 'tfjs_model')
print('✅ Model converted to TF.js format')

# Show output files
print('\n📁 Output files:')
for f in os.listdir('tfjs_model'):
    size = os.path.getsize(f'tfjs_model/{f}') / 1024
    print(f'  {f} ({size:.1f} KB)')

# Zip for download
shutil.make_archive('tfjs_model', 'zip', 'tfjs_model')
print('\n📦 Zipped and ready to download!')

## Step 6: Download the Model

In [ ]:
from google.colab import files
files.download('tfjs_model.zip')
print('\n🎉 Done! Send this zip to Little Alpha and the web app will work with real predictions!')